<h3>Multinomial Naive Bayes</h3>

In [3]:
function fit(X,Y)
    --calculate log posterior
    local cc = torch.zeros(1, Y:size(2))
    local fc = torch.zeros(Y:size(2), X:size(2))
    local clp = torch.zeros(1, Y:size(2))
    
    fc:addcmul(Y:t(),(X))
    cc:add(Y:sum(1))
    
    --smoothing
    fc:add(1)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --local total = cc:sum(2)
    --cc:div(total[1][1])
    --clp = cc:log()
    --total:log()
    --cc:log()
    --cc:csub(total[1][1])
    
    --this is just a dumb prior but works better than above?!
    clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), clp --:csub(math.log(Y:size(2)))
end

function predict(X, W, b)
    --joint log-likelihood
    return ((X*W:t()):add(b:expand(b,X:size(1),b:size(2))))
end

function predict_score(ypred, ytrue)
    local c = 0
    for i=1,ypred:size(1) do
        if ypred[i][1] == ytrue[i][1] then
            c = c + 1       
        end
    end
    return c/ypred:size(1)
end

<h3>Document by Word Tensor</h3>

In [4]:
--feature weight: counts
function createDocWordMatrix(vocab, max_sent_len, sparseMatrix)
    docword = torch.zeros(sparseMatrix:size(1), vocab)
    for i=1,sparseMatrix:size(1) do
        for j=1, max_sent_len do
            local idx = (sparseMatrix[i][j])
            if idx ~= 0 then
                docword[i][idx] = 1 --+ docword[i][idx]
            end
        end
    end
    return docword
end
 

<h3>One-Hot Encoding</h3>

In [5]:
function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

<h3>Write to file</h3>

In [6]:
function write2file(fname, pred) 
    f = io.open(fname, "w")
    f:write("ID,Category\n")
    for i=1,pred:size(1) do
        f:write(tostring(i) .. "," .. tostring(pred[i][1]) .. "\n")
    end
    f:close()
end

<h3>Reading in HDF5 files for SST1</h3>

In [7]:
require('hdf5')

f = hdf5.open("SST1.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
X_test = f:read("test_input"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

In [18]:
--for final testing ONLY!
--remember to also change var names when creating docword matrix
X_train = torch.cat(X_train, X_valid, 1)
Y_train = torch.cat(Y_train, Y_valid, 1)

In [8]:
X_train =createDocWordMatrix(nfeatures, 53, X_train)
Y_train = onehotencode(nclasses, Y_train)
X_test = createDocWordMatrix(nfeatures, 53, X_valid)
Y_valid = onehotencode(nclasses, Y_valid)

In [ ]:
--fit MNB
lp, clp = fit(X_train, Y_train)
Y_pred = predict(X_test, lp, clp)
_, Y_pred = torch.max(Y_pred, 2)
_,Y_true = torch.max(Y_valid, 2)
acc_score = predict_score(Y_pred, Y_true)
print(acc_score)

In [21]:
write2file("MNB_3.csv", Y_pred)

In [27]:
lp, clp = fit(X_train, Y_train)
Y_pred = predict(X_test, lp, clp)
_, Y_pred = torch.max(Y_pred, 2)
_,Y_true = torch.max(Y_valid, 2)
acc_score = predict_score(Y_pred, Y_true)
print(acc_score)

0.39146230699364	
